In [11]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt

In [132]:
def f(x,y):
    return 418.9829*2 - x*np.sin(np.sqrt(abs(x))) - y*np.sin(np.sqrt(abs(y)))

In [50]:
f.__code__.co_argcount

2

### Estructura cromosómica

In [133]:
class Gen:
    def __init__(self,dominio,precision):
        """
        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión.
        """       
        self.dominio = dominio
        self.precision = precision
        self.len = self._length()

        # Offset all(0) -> lb, all(1) -> ub
        self.offinf = self.dominio[0]
        self.offsup = self.translate('1'*self.len) - (self.dominio[1]-self.dominio[0]) ##### UC

    def _length(self):
        """Calcula el número de bits en un gen.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = self.dominio[1] - self.dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**self.precision - 1))
        return cell_count
    
    def translate(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) genes. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''
        offinf = Decimal(str(self.offinf))

        if mode == 'code':
            ## Real -> Binario
            
            # Cortar decimales por redondeo
            x = float(x) # failsafe
            x = round(x,self.precision)

            # Offset - Signo -> rango>=0
            x = Decimal(str(x))
            x += abs(offinf) + abs() ##### UC
            
            # Eliminar punto
            x_int,x_dec = str(x).split(".")
            x_dec = x_dec.ljust(self.precision,"0") # fill precision

            # Representación entera
            x_r = int(x_int + x_dec)
            return np.binary_repr(x_r, width=self.len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = str(int(x,2))

            # Añadir el punto decimal
            int_part = x[:-self.precision]
            dec_part = x[-self.precision:]
            x_float = Decimal(int_part + '.' + dec_part)

            # Extraer offset
            x_float -= abs(offinf)
            return float(x_float)

In [137]:
class Cromosoma:
    def __init__(self,genes):
        """
        Parameters
        ----------
        genes : lsit,tuple
            Genes del cromosoma ordenados.
        """       
        self.genes = genes
        self.len = sum([gen.len for gen in genes])

    def representacion(self,x,mode):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas a través de sus genes.

        Parameters
        ----------
        x : list o str
            Expresión del cromosoma (lista si los genes se expresan en números reales, str si el cromosoma es binario)
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str o list
            Cromosoma traducido (elementos tipo str si "genes" era una lista, lista si "genes" era binario).
        '''
        
        # liminf = self.dominio[0]
        # liminf = Decimal(str(liminf))

        if mode == 'code':
            ## Real -> Binario
            genes_bin = [gen.translate(val, mode=mode) for val,gen in zip(x,self.genes)]
            self.cromosoma = ''.join(genes_bin)
            return self.cromosoma
        
        elif mode == 'decode':
            ## Binario -> Real
            genes_lens = [gen.len for gen in self.genes]
            ind_final = np.cumsum(genes_lens)
            genes_r = [gen.translate(x[ind-lens:ind]) for gen, lens, ind in zip(self.genes, genes_lens, ind_final)]
            return genes_r

### Generación aleatoria

In [143]:
Gen1 = Gen((-500,500),3)
Gen2 = Gen((-500,500),5)
CromA = Cromosoma(genes=(Gen1,Gen2))

47

In [153]:
N = 10
crom_len = CromA.len
pp = np.random.binomial(n=1, p=0.5, size=(N,crom_len))

pp = pp.astype(str)
pp

array([['1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1',
        '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0',
        '1', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1',
        '0', '1', '0', '1', '0', '1', '1', '1'],
       ['1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1',
        '0', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1',
        '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1',
        '0', '1', '1', '1', '1', '0', '1', '0'],
       ['0', '1', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '1',
        '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1',
        '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '1', '1',
        '1', '1', '0', '0', '1', '0', '0', '0'],
       ['1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1',
        '1', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '0', '1',
        '1', '0', '0', '1', '1', '1', '0', '0', '0

In [170]:
pp_bin = np.array([''.join(xi) for xi in pp])
pp_bin

array(['10001000001110000010110000100000111100101010111',
       '10001101110110101110111111000001011111101111010',
       '01101110100111100000011111010100001011111001000',
       '10011000101111010101110101100111000111110000000',
       '00111011000101111001010011011111011001100010111',
       '01010000101011110011101000111111001010010101110',
       '00001010111011111010101000010011000010110010000',
       '01000000000001111110101001010110101010101110011',
       '01001111000100100010111011001011110101111111110',
       '10111110010110000010011010101011101101101100011'], dtype='<U47')

In [171]:
np.array([CromA.representacion(cr,mode='decode') for cr in pp_bin])

array([[  57.954  ,  517.42935],
       [  81.038  ,  821.69594],
       [ -46.912  ,  156.7316 ],
       [ 125.621  ,  624.30976],
       [-257.959  ,  -91.25225],
       [-169.517  ,  359.55758],
       [-455.206  ,  345.10096],
       [-237.73   ,  367.26003],
       [-176.126  ,  741.20062],
       [ 279.65   ,   59.58371]])

In [172]:
CromA.representacion('1'*CromA.len,mode='decode')

[548.575, 842.17727]

In [ ]:
def random_generator(N,size):
   '''
   Rutina de generación aleatoria de individuos

   Parameters
   ----------
   N : int
      Número de individuos en la población.
   size : int
      Longitud del cromosoma característico de los individuos en la población.

   Returns
   -------
   tuple (array,array)
      Array de cromosomas generados y un array con la aptitud de cada individuo.
   '''
   poblacion = np.random.binomial(n=1, p=0.5, size=(N,size))
   

In [56]:
class SimpleEvolutionary:
    # Constructor
    def __init__(self,f,dominios,precisiones):
        """
        Parameters
        ----------
        f : function
            Función a optimizar por el algoritmo.
        dominios : tuple, list
            Iterable que contiene tuplas (liminf, limsup) de los dominios de las variables para la función "f".
        precisiones : tuple, list
            Iterable que contiene el número (int) de cifras decimales luego del punto para cada variable.
        """
        self.f = f
        self.dominios = dominios
        self.precisiones = precisiones

        # Lóngitudes de genes y cromosoma
        self.gen_lens = [self.gen_length(dom,prc) for dom,prc in zip(dominios,precisiones)]
        self.chrom_len = sum(self.gen_lens)
    
    ########################
    #### REPRESENTACIÓN ####
    def gen_length(self,dominio,precision):
        """Calcula el número de bits en un gen.

        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = dominio[1] - dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**precision - 1))
        return cell_count
    
    def gen_translate(self,x,locus,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        locus : int
            Número de gen en el cromosoma al que pertenece.
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''

        if mode == 'code':
            ## Real -> Binario

            x = Decimal(str(x))
            liminf = self.dominios[locus][0]
            liminf = Decimal(str(liminf))
            # Signo -> trasladar a rango>=0
            x += abs(liminf)
            # Eliminar punto
            x_int = str(x).replace(".","")
            return np.binary_repr(x_int, width=self.chrom_len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = str(int(x,2))
            # Añadir el punto decimal
            int_part = x[:-self.precision[locus]]
            dec_part = x[self.precision[locus]:]
            return float(int_part + '.' + dec_part)

In [102]:
x = 486.1345678
str(Decimal(str(x)))

'486.1345678'

In [103]:
x = -486.1345678
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = int(str(x).replace(".",""))
print(x_int)
np.binary_repr(x_int)

-486.1345678
-500
13.8654322
138654322


'1000010000111011001001110010'

In [92]:
int(np.binary_repr(x_int),2)

98613478

In [87]:

presicion = 3

x = str(int(x,2))


# Añadir el punto decimal
int_part = x[:-self.precision[locus]]
dec_part = x[self.precision[locus]:]
return float(int_part + '.' + dec_part)

'abcd'

In [70]:
".".join(('312','009'))

'312.009'

In [94]:
## Real -> Binario
x = -421.849878
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = str(x).replace(".","")
print(x_int)
# np.binary_repr(x_int, width=)

-421.849878
-500
78.150122
78150122


### Representación: Transformando reales



### Población: Generación aleatoria

In [5]:
def aptitud(binary):
    '''
    Traduce las entradas binarias a entradas reales para ser evaluadas en la función objetivo "f".
    '''
